In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm, trange
tqdm.pandas()

import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from numba import jit, njit

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### Import data

In [2]:
df = pd.read_csv('../data/wikihowAll.csv', delimiter=',', nrows=10000)

In [3]:
df.shape

(10000, 3)

Датасет дуже великий і його обробка займає дуже багато часу, тому візьму перші 1000 рядків. В майбутньому збільшу кількість.

In [4]:
df = df[:1000]

In [5]:
df.shape

(1000, 3)

In [6]:
df.head()

,headline,title,text
0,"\nKeep related supplies in the same area.,\nMa...",How to Be an Organized Artist1,"If you're a photographer, keep all the necess..."
1,\nCreate a sketch in the NeoPopRealist manner ...,How to Create a Neopoprealist Art Work,See the image for how this drawing develops s...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",How to Be a Visual Effects Artist1,It is possible to become a VFX artist without...
3,\nStart with some experience or interest in ar...,How to Become an Art Investor,The best art investors do their research on t...
4,"\nKeep your reference materials, sketches, art...",How to Be an Organized Artist2,"As you start planning for a project or work, ..."


In [7]:
print(df['headline'][0])


Keep related supplies in the same area.,
Make an effort to clean a dedicated workspace after every session.,
Place loose supplies in large, clearly visible containers.,
Use clotheslines and clips to hang sketches, photos, and reference material.,
Use every inch of the room for storage, especially vertical space.,
Use chalkboard paint to make space for drafting ideas right on the walls.,
Purchase a label maker to make your organization strategy semi-permanent.,
Make a habit of throwing out old, excess, or useless stuff each month.


In [8]:
print(df['text'][0])

 If you're a photographer, keep all the necessary lens, cords, and batteries in the same quadrant of your home or studio. Paints should be kept with brushes, cleaner, and canvas, print supplies should be by the ink, etc. Make broader groups and areas for your supplies to make finding them easier, limiting your search to a much smaller area. Some ideas include:


Essential supplies area -- the things you use every day.
Inspiration and reference area.
Dedicated work area .
Infrequent or secondary supplies area, tucked out of the way.;
, This doesn't mean cleaning the entire studio, it just means keeping the area immediately around the desk, easel, pottery wheel, etc. clean each night. Discard trash or unnecessary materials and wipe down dirty surfaces. Endeavor to leave the workspace in a way that you can sit down the next day and start working immediately, without having to do any work or tidying.


Even if the rest of your studio is a bit disorganized, an organized workspace will help 

Можна вважати вміст колонки 'headline' підсумованим текстом колонки 'text'. В майбутньому можна використати для оцінки побудованої моделі. Колонку 'title' я не використовуватиму, то її видалю.

In [9]:
df.drop('title', inplace=True, axis=1)

In [10]:
df.shape

(1000, 2)

In [11]:
df['text'].isnull().sum()

4

Чотири рядки мають значення Nan тому їх теж треба видалити

In [12]:
df.dropna(inplace=True)

In [13]:
df['text'].isnull().sum()

0

In [14]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [16]:
def clean_text(texts):
    corpus = []
    
    for t in tqdm(texts):
        text = t.lower()
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
            text = " ".join(new_text)
        text = re.sub(r"(@[a-z]+)|([^a-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
        text = re.sub(r"\d+", "", text)
        text = text.split()
        te = []
        for word in text:
            if word not in stop_words:
                te.append(word)
        te = ' '.join(te)
        corpus.append(te)
        
    return corpus

In [17]:
sentences = df['text'].progress_apply(lambda x: x).values

In [18]:
df['text'] = clean_text(sentences)

In [19]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [20]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [21]:
def word_lemmatizer(texts):
    corpus = []
    lemmatizer = WordNetLemmatizer()
    for text in tqdm(texts):
        t = []
        for i in word_tokenize(text):            
            lem_text = lemmatizer.lemmatize(i, get_wordnet_pos(i))
            t.append(lem_text)
        t = ' '.join(t)
        corpus.append(t)
    return corpus

In [22]:
sentences = df['text'].progress_apply(lambda x: x).values

In [ ]:
df['text'] = word_lemmatizer(sentences)

In [ ]:
df.head()

In [ ]:
def clean_text_summary(texts):
    corpus = []
    
    for t in tqdm(texts):
        text = t.lower()
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
            text = " ".join(new_text)
        text = re.sub(r"(@[A-Za-z]+)|([^A-za-z \t])", "", text)
        text = re.sub(r"\d+", "", text)
        
        corpus.append(text)
        
    return corpus

In [ ]:
sentences = df['headline'].progress_apply(lambda x: x).values
df['headline'] = clean_text_summary(sentences)

In [ ]:
df.head()

In [ ]:
df['text'].replace('', np.nan, inplace=True)

In [ ]:
df['text'].isnull().sum()

In [ ]:
df.to_csv(r'../data/wikihowAll_cleaned.csv', index=False)